In [611]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import regex as re
import math
from collections import defaultdict

from rapidfuzz.fuzz import ratio

things to look at: how accurate is the prematch probability when it ends (if a team has over 50% chance to win, how often do they actually win the game?)

if my fair probability is wildely different than the midprice, should I weigh it less or find a new fair probability? How often is my direction right?

can i still find edge in a very wide market indicate?

In [612]:
date = '2025-11-24'
odds_sport = 'cbb' #cbb, cfb, nba, nfl
kalshi_sport = 'ncaab' #ncaab, ncaaf, nba, nfl

In [613]:
odds_df = pd.read_csv(f"../data_collection/updated_scripts/oddsapi_outputs/{date}/{odds_sport}_odds.csv")
odds_df.drop(columns=['league'], inplace=True)
odds_df.rename(columns={'price': 'odds'}, inplace=True)
odds_df['vig_prob'] = 1 / odds_df['odds']

def remove_vig_probs(df):
    df = df.copy()
    df['fair_prb'] = pd.NA

    grouped = df.groupby(['game_id', 'bookmaker', 'market'])

    for _, group in grouped:
        if len(group) < 2:
            continue
        probs = group['vig_prob']
        total = probs.sum()
        if total == 0:
            continue
        fair_probs = (probs / total).round(4)
        df.loc[group.index, 'fair_prb'] = fair_probs

    return df

odds_df = remove_vig_probs(odds_df)


odds_winners_df = odds_df[odds_df['market'] == 'h2h'].copy()
odds_spreads_df = odds_df[odds_df['market'] == 'spreads'].copy()
odds_spreads_df = odds_spreads_df.loc[(odds_spreads_df['point'].notna()) & (odds_spreads_df['point'] > 0)]
odds_totals_df  = odds_df[odds_df['market'] == 'totals'].copy()

# Average per-team fair probabilities across DraftKings/FanDuel/Pinnacle for winners_df
mask = odds_winners_df['fair_prb'].notna()
avg_by_team = (
    odds_winners_df.loc[mask]
    .groupby(['game_id', 'team'])['fair_prb']
    .transform('median')
    .round(4)
)
odds_winners_df.loc[mask, 'avg_fair_prb'] = avg_by_team
odds_winners_df.loc[~mask, 'avg_fair_prb'] = pd.NA

#Average fair probabilities for spreads for same game, point spread, and team
mask = odds_spreads_df['fair_prb'].notna()
avg_by_point = (
    odds_spreads_df.loc[mask]
    .groupby(['game_id', 'point', 'team'])['fair_prb']
    .transform('mean')
    .round(4)
)
odds_spreads_df['avg_fair_prb'] = avg_by_point

#Average fair probabilities for totals for same game, point spread, direction (Over/Under)
mask = odds_totals_df['fair_prb'].notna()
avg_by_tot_point = (
    odds_totals_df.loc[mask]
    .groupby(['game_id', 'point', 'team'])['fair_prb']
    .transform('mean')
    .round(4)
)
odds_totals_df['avg_fair_prb'] = avg_by_tot_point

In [614]:
kalshi_winners_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_winners.csv")
kalshi_totals_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_totals.csv")
kalshi_spreads_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_spreads.csv")

if kalshi_sport == 'ncaaf':
    kalshi_spreads_df['points'] = kalshi_spreads_df['title'].str.extract(r'over ([\d.]+) points\?').astype(float)
    kalshi_totals_df["points"] = kalshi_totals_df["ticker"].str.extract(r"-([0-9.]+)$").astype(float)
elif (kalshi_sport == 'ncaab') | (kalshi_sport == 'nba'):
    kalshi_spreads_df['points'] = kalshi_spreads_df['title'].str.extract(r'over ([\d.]+) Points\?').astype(float)
    kalshi_totals_df["points"] = kalshi_totals_df["ticker"].str.extract(r"-([0-9.]+)$").astype(float)
    



columns_to_drop = ['timestamp', 'market_type', 'yes_bid2', 'yes_ask2', 'no_bid2', 'no_ask2', 'yes_depth_bids', 'yes_depth_asks', 'no_depth_bids', 'no_depth_asks']
kalshi_winners_df.drop(columns=columns_to_drop, inplace=True)
kalshi_spreads_df.drop(columns=columns_to_drop, inplace=True)
kalshi_totals_df.drop(columns=columns_to_drop, inplace=True)

In [615]:
odds_totals_df.head()

,sport,game_id,start_time,bookmaker,market,team,odds,point,home_team,away_team,vig_prob,fair_prb,avg_fair_prb
10,CBB,472610dfc22034c972a32525e329f6af,2025-11-24 11:00:00 CST,BetMGM,totals,Over,1.67,130.5,Bowling Green Falcons,Bucknell Bison,0.598802,0.5628,0.5628
11,CBB,472610dfc22034c972a32525e329f6af,2025-11-24 11:00:00 CST,BetMGM,totals,Under,2.15,130.5,Bowling Green Falcons,Bucknell Bison,0.465116,0.4372,0.4372
22,CBB,4cc31cb49aa33f6f07b1b8fadd072568,2025-11-24 11:04:49 CST,BetMGM,totals,Over,1.87,136.5,Fairleigh Dickinson Knights,East Texas A&M Lions,0.534759,0.5,0.5
23,CBB,4cc31cb49aa33f6f07b1b8fadd072568,2025-11-24 11:04:49 CST,BetMGM,totals,Under,1.87,136.5,Fairleigh Dickinson Knights,East Texas A&M Lions,0.534759,0.5,0.5
28,CBB,a3ca87da634c46067cae74a0960296ef,2025-11-24 11:30:00 CST,FanDuel,totals,Over,1.98,152.5,Oregon St Beavers,Massachusetts Minutemen,0.505051,0.472,0.472


In [616]:
#get names from kalshi_winners_df
def extract_teams_from_winners(title):
    title = title.replace(" Winner?", "")
    if " at " in title:
        right, left = title.split(" at ", 1)
    elif " vs " in title:
        right, left = title.split(" vs ", 1)
    else:
        return pd.Series([None, None])  
    left = re.sub(r'\bSt\.$', 'St', left.strip())
    right = re.sub(r'\bSt\.$', 'St', right.strip())
    return pd.Series([left, right])

kalshi_winners_df[['home_team', 'away_team']] = kalshi_winners_df['title'].apply(extract_teams_from_winners)
unique_rows = kalshi_winners_df.drop_duplicates(subset=['home_team', 'away_team'])
flat_teams = pd.unique(unique_rows[['home_team', 'away_team']].values.ravel())
kalshi_winners_teams = flat_teams.tolist()

#get names from kalshi_totals_df
def extract_teams_from_totals(title):
    title = title.replace(": Total Points", "")
    if " at " in title:
        right, left = title.split(" at ", 1)
        left = re.sub(r'\bSt\.$', 'St', left.strip())
        right = re.sub(r'\bSt\.$', 'St', right.strip())
        return pd.Series([left, right])
    return None

kalshi_totals_df[['home_team', 'away_team']] = kalshi_totals_df['title'].apply(extract_teams_from_totals)
unique_rows = kalshi_winners_df.drop_duplicates(subset=['home_team', 'away_team'])
flat_teams = pd.unique(unique_rows[['home_team', 'away_team']].values.ravel())
kalshi_totals_teams = flat_teams.tolist()

#get names from kalshi_spreads_df
def extract_team_from_spreads(title):
    if " wins by " in title:
        team = title.split(" wins by ", 1)[0].strip()
        team = re.sub(r'\bSt\.$', 'St', team)
        return team
    return None

kalshi_spreads_df['team'] = kalshi_spreads_df['title'].apply(extract_team_from_spreads)
unique_teams_spread = kalshi_spreads_df['team'].drop_duplicates()
kalshi_spreads_teams = unique_teams_spread.tolist()

In [617]:
odds_teams_by_market = odds_df.groupby('market')['team'].unique().to_dict()

def fuzzy_match_kalshi_to_odds(kalshi_teams, odds_team_names):
    matched_kalshi = []
    matched_odds = []
    candidates_dict = defaultdict(list)

    kalshi_sorted = sorted(kalshi_teams, key=lambda x: x[0] if x else '')
    remaining_odds = sorted(odds_team_names.tolist().copy())

    for kalshi_name in kalshi_sorted:
        candidates = []
        for odds_name in remaining_odds:
            if kalshi_name in odds_name:
                candidates.append(odds_name)
        if len(candidates) == 1:
            candidates_dict[candidates[0]].append(kalshi_name)
        elif len(candidates) > 1:
            best_fit = candidates[0]
            best_ratio = ratio(best_fit, kalshi_name)
            for name in candidates:
                curr_ratio = ratio(name, kalshi_name)
                if curr_ratio > best_ratio:
                    best_fit = name
                    best_ratio = curr_ratio
            candidates_dict[best_fit].append(kalshi_name)
    
    for odd, kalsh in candidates_dict.items():
        best_fit = kalsh[0]
        best_ratio = ratio(best_fit, odd)
        if len(kalsh) > 1:
            for name in kalsh:
                curr_ratio = ratio(name, odd)
                if curr_ratio > best_ratio:
                    best_fit = name
                    best_ratio = curr_ratio
        matched_odds.append(odd)
        matched_kalshi.append(best_fit)


    return matched_kalshi, matched_odds


# Winners / h2h
matched_kalshi_h2h, matched_odds_h2h = fuzzy_match_kalshi_to_odds(
    kalshi_winners_teams,
    odds_teams_by_market.get('h2h', [])
)

# Spreads
matched_kalshi_spreads, matched_odds_spreads = fuzzy_match_kalshi_to_odds(
    kalshi_spreads_teams,
    odds_teams_by_market.get('spreads', [])
)

# Totals (match only Over/Under)
totals_odds_df = odds_df[odds_df['market'] == 'totals']
odds_totals_teams = pd.unique(totals_odds_df[['home_team', 'away_team']].values.ravel())
matched_kalshi_totals, matched_odds_totals = fuzzy_match_kalshi_to_odds(
    kalshi_totals_teams,
    odds_totals_teams
)

matched_names = {
    'h2h': {
        'kalshi': matched_kalshi_h2h,
        'odds': matched_odds_h2h
    },
    'spreads': {
        'kalshi': matched_kalshi_spreads,
        'odds': matched_odds_spreads
    },
    'totals': {
        'kalshi': matched_kalshi_totals,
        'odds': matched_odds_totals
    }
}


In [618]:
assert(len(matched_names['h2h']['kalshi']) == len(matched_names['h2h']['odds']))
assert(len(matched_names['spreads']['kalshi']) == len(matched_names['spreads']['odds']))
assert(len(matched_names['totals']['kalshi']) == len(matched_names['totals']['odds']))

In [619]:
odds_winners_df = odds_winners_df[
    odds_winners_df['home_team'].isin(matched_names['h2h']['odds']) |
    odds_winners_df['away_team'].isin(matched_names['h2h']['odds'])
].drop_duplicates(subset='team').sort_values(by='home_team').reset_index(drop=True)

kalshi_winners_df = kalshi_winners_df[
    kalshi_winners_df['home_team'].isin(matched_names['h2h']['kalshi']) |
    kalshi_winners_df['away_team'].isin(matched_names['h2h']['kalshi'])
].sort_values(by='home_team').reset_index(drop=True)

odds_spreads_df = odds_spreads_df[odds_spreads_df['team'].isin(matched_names['spreads']['odds'])].reset_index(drop=True)
kalshi_spreads_df = kalshi_spreads_df[kalshi_spreads_df['team'].isin(matched_names['spreads']['kalshi'])].reset_index(drop=True)

odds_totals_df = odds_totals_df[
    odds_totals_df['home_team'].isin(matched_names['totals']['odds']) |
    odds_totals_df['away_team'].isin(matched_names['totals']['odds'])
].sort_values(by='home_team').reset_index(drop=True)
kalshi_totals_df = kalshi_totals_df[
    (kalshi_totals_df['home_team'].isin(matched_names['totals']['kalshi'])) | 
    (kalshi_totals_df['away_team'].isin(matched_names['totals']['kalshi']))
    ].sort_values(by='home_team').reset_index(drop=True)


In [620]:
odds_winners_df

,sport,game_id,start_time,bookmaker,market,team,odds,point,home_team,away_team,vig_prob,fair_prb,avg_fair_prb
0,CBB,4090c3df00d479dd1dd04d715c12b668,2025-11-24 20:00:00 CST,DraftKings,h2h,Abilene Christian Wildcats,1.77,NaN,Abilene Christian Wildcats,UTSA Roadrunners,0.564972,0.5426,0.5501
1,CBB,4090c3df00d479dd1dd04d715c12b668,2025-11-24 20:00:00 CST,DraftKings,h2h,UTSA Roadrunners,2.10,NaN,Abilene Christian Wildcats,UTSA Roadrunners,0.476190,0.4574,0.4499
2,CBB,5e48198f3d109d3e1fec080d4be32438,2025-11-24 21:30:00 CST,DraftKings,h2h,Denver Pioneers,21.00,NaN,Arizona Wildcats,Denver Pioneers,0.047619,0.0463,0.0463
3,CBB,5e48198f3d109d3e1fec080d4be32438,2025-11-24 21:30:00 CST,DraftKings,h2h,Arizona Wildcats,1.02,NaN,Arizona Wildcats,Denver Pioneers,0.980392,0.9537,0.9537
4,CBB,a74601254aebd00fb7d108a09acbc2ec,2025-11-24 19:00:00 CST,DraftKings,h2h,Auburn Tigers,1.59,NaN,Auburn Tigers,Oregon Ducks,0.628931,0.6035,0.6035
5,CBB,a74601254aebd00fb7d108a09acbc2ec,2025-11-24 19:00:00 CST,DraftKings,h2h,Oregon Ducks,2.42,NaN,Auburn Tigers,Oregon Ducks,0.413223,0.3965,0.3965
6,CBB,9ba4f1c8078513678e7474ddaad2e137,2025-11-24 16:00:00 CST,FanDuel,h2h,USC Trojans,1.36,NaN,Boise State Broncos,USC Trojans,0.735294,0.6916,0.6904
7,CBB,9ba4f1c8078513678e7474ddaad2e137,2025-11-24 16:00:00 CST,FanDuel,h2h,Boise State Broncos,3.05,NaN,Boise State Broncos,USC Trojans,0.327869,0.3084,0.3096
8,CBB,16064389f68662085185f157040a8531,2025-11-24 13:06:13 CST,DraftKings,h2h,Baylor Bears,1.45,NaN,Creighton Bluejays,Baylor Bears,0.689655,0.6506,0.637
9,CBB,16064389f68662085185f157040a8531,2025-11-24 13:06:13 CST,DraftKings,h2h,Creighton Bluejays,2.70,NaN,Creighton Bluejays,Baylor Bears,0.370370,0.3494,0.363


In [621]:
# Concatenate winners df

# Specify the columns to extract
kalshi_cols = ['ticker', 'yes_bid', 'yes_ask', 'home_team', 'away_team']
odds_cols = ['market', 'start_time', 'team', 'home_team', 'away_team', 'avg_fair_prb']

# Rename overlapping columns in odds to prevent clashes
odds_subset = odds_winners_df[odds_cols].rename(columns={
    'home_team': 'odds_home_team',
    'away_team': 'odds_away_team'
})

kalshi_subset = kalshi_winners_df[kalshi_cols].rename(columns={
    'home_team': 'kalshi_home_team',
    'away_team': 'kalshi_away_team'
})

combined_rows = []
len_matched = matched_names['h2h']['kalshi']
matched_names_h2h = matched_names['h2h']

for i in range(len(len_matched)):
    odds_name = matched_names_h2h['odds'][i]
    kalshi_name = matched_names_h2h['kalshi'][i]

    # Find the corresponding odds row
    odds_row = odds_subset.loc[odds_subset['team'] == odds_name]
    assert len(odds_row) == 1, f"Expected one row for {odds_name}, got {len(odds_row)}"

    # Find the two matching Kalshi rows
    kalshi_rows = kalshi_subset.loc[
        (kalshi_subset['kalshi_home_team'] == kalshi_name) |
        (kalshi_subset['kalshi_away_team'] == kalshi_name)
    ]
    assert len(kalshi_rows) == 2, f"Expected two rows for {kalshi_name}, got {len(kalshi_rows)}"

    # Extract rows
    k1 = kalshi_rows.iloc[0]
    k2 = kalshi_rows.iloc[1]
    midprice1 = (k1['yes_bid'] + k1['yes_ask']) / 2
    midprice2 = (k2['yes_bid'] + k2['yes_ask']) / 2

    # Extract scalar fair probability
    prb = odds_row['avg_fair_prb'].astype(float).item()

    # Choose the row closer to the odds probability
    if ((midprice1 - prb) ** 2) < ((midprice2 - prb) ** 2):
        combined_row = pd.concat([k1, odds_row.iloc[0]])
    else:
        combined_row = pd.concat([k2, odds_row.iloc[0]])

    combined_rows.append(combined_row)

combined_winners_df = pd.DataFrame(combined_rows)
combined_winners_df = combined_winners_df.reset_index(drop=True)

In [622]:
EDGE = 0.01
KELLY_UPPERBOUND = 0.25
BANKROLL = 200.00
Q1_WEIGHT = 1.00
Q2_WEIGHT = 1.00
Q3_WEIGHT = 1.00
Q4_WEIGHT = 1.00

edge_winners_df = combined_winners_df.loc[(combined_winners_df['avg_fair_prb'] >= combined_winners_df['yes_ask'] + EDGE) |
                        (combined_winners_df['avg_fair_prb'] <= combined_winners_df['yes_bid'] - EDGE)].reset_index(drop=True)

midprice = (edge_winners_df['yes_bid'] + edge_winners_df['yes_ask']) / 2

edge_winners_df['raw_kelly'] = (
    (edge_winners_df['avg_fair_prb'] - midprice) /
    (1 - midprice)
).clip(lower=0)

total_kelly = edge_winners_df['raw_kelly'].sum()
edge_winners_df['raw_kelly'] = pd.DataFrame({
    'original': edge_winners_df['raw_kelly'],
    'normalized': (edge_winners_df['raw_kelly'] / total_kelly)
}).min(axis=1)

# Define the real_kelly logic
def scale_kelly(row):
    k = row['raw_kelly']
    p = row['avg_fair_prb']
    
    if k == 0 or pd.isna(k):
        return 0
    if 0.1 <= p < 0.25:
        return min(Q1_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.25 <= p < 0.5:
        return min(Q2_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.5 <= p < 0.75:
        return min(Q3_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.75 <= p < 0.9:
        return min(Q4_WEIGHT * k, KELLY_UPPERBOUND)
    else:
        return 0  # fallback if out of range

# Apply to the DataFrame
edge_winners_df['real_kelly'] = edge_winners_df.apply(scale_kelly, axis=1)
edge_winners_df['optimal_bet'] = edge_winners_df['real_kelly'] * BANKROLL

num_contracts = edge_winners_df['optimal_bet'] // edge_winners_df['yes_bid']
edge_winners_df['num_contracts'] = num_contracts
trading_cost = np.ceil(0.0175 * num_contracts * edge_winners_df['yes_bid'] * (1 - edge_winners_df['yes_bid']))
edge_winners_df['trading_cost'] = trading_cost
profit = (1 - edge_winners_df['yes_bid']) *  num_contracts - trading_cost
edge_winners_df['profit'] = profit
edge_winners_df['ev'] = (profit * edge_winners_df['avg_fair_prb'] - edge_winners_df['optimal_bet'] * (1 - edge_winners_df['avg_fair_prb'])).round(2)
filtered_winners_df = edge_winners_df.loc[edge_winners_df['ev'] > 0].reset_index(drop=True)

In [623]:
edge_winners_df

,ticker,yes_bid,yes_ask,kalshi_home_team,kalshi_away_team,market,start_time,team,odds_home_team,odds_away_team,avg_fair_prb,raw_kelly,real_kelly,optimal_bet,num_contracts,trading_cost,profit,ev
0,KXNCAAMBGAME-25NOV24ASUTEX-ASU,0.26,0.27,Texas,Arizona St,h2h,2025-11-24 22:30:00 CST,Arizona St Sun Devils,Texas Longhorns,Arizona St Sun Devils,0.2885,0.031973,0.031973,6.394558,24.0,1.0,16.76,0.29
1,KXNCAAMBGAME-25NOV24DENARIZ-ARIZ,0.99,1.00,Arizona,Denver,h2h,2025-11-24 21:30:00 CST,Arizona Wildcats,Arizona Wildcats,Denver Pioneers,0.9537,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00
2,KXNCAAMBGAME-25NOV24CREIBAY-BAY,0.58,0.60,Baylor,Creighton,h2h,2025-11-24 13:06:13 CST,Baylor Bears,Creighton Bluejays,Baylor Bears,0.6370,0.114634,0.114634,22.926829,39.0,1.0,15.38,1.47
3,KXNCAAMBGAME-25NOV24CREIBAY-CREI,0.41,0.42,Baylor,Creighton,h2h,2025-11-24 13:06:13 CST,Creighton Bluejays,Creighton Bluejays,Baylor Bears,0.3630,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00
4,KXNCAAMBGAME-25NOV24DENARIZ-DEN,0.00,0.01,Arizona,Denver,h2h,2025-11-24 21:30:00 CST,Denver Pioneers,Arizona Wildcats,Denver Pioneers,0.0463,0.041508,0.000000,0.000000,NaN,NaN,NaN,NaN
5,KXNCAAMBGAME-25NOV24OHIOGMU-GMU,0.74,0.75,George Mason,Ohio,h2h,2025-11-24 17:00:00 CST,George Mason Patriots,George Mason Patriots,Ohio Bobcats,0.7261,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00
6,KXNCAAMBGAME-25NOV24KENNRICE-KENN,0.65,0.67,Rice,Kennesaw St,h2h,2025-11-24 15:30:00 CST,Kennesaw St Owls,Rice Owls,Kennesaw St Owls,0.6386,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00
7,KXNCAAMBGAME-25NOV24KUND-KU,0.68,0.69,Notre Dame,Kansas,h2h,2025-11-24 14:30:00 CST,Kansas Jayhawks,Kansas Jayhawks,Notre Dame Fighting Irish,0.6589,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00
8,KXNCAAMBGAME-25NOV24MICHSDSU-MICH,0.70,0.71,San Diego St,Michigan,h2h,2025-11-24 21:30:00 CST,Michigan Wolverines,Michigan Wolverines,San Diego St Aztecs,0.6831,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00
9,KXNCAAMBGAME-25NOV24KUND-ND,0.32,0.33,Notre Dame,Kansas,h2h,2025-11-24 14:30:00 CST,Notre Dame Fighting Irish,Kansas Jayhawks,Notre Dame Fighting Irish,0.3411,0.023852,0.023852,4.770370,14.0,1.0,8.52,-0.24


In [624]:
filtered_winners_df

,ticker,yes_bid,yes_ask,kalshi_home_team,kalshi_away_team,market,start_time,team,odds_home_team,odds_away_team,avg_fair_prb,raw_kelly,real_kelly,optimal_bet,num_contracts,trading_cost,profit,ev
0,KXNCAAMBGAME-25NOV24ASUTEX-ASU,0.26,0.27,Texas,Arizona St,h2h,2025-11-24 22:30:00 CST,Arizona St Sun Devils,Texas Longhorns,Arizona St Sun Devils,0.2885,0.031973,0.031973,6.394558,24.0,1.0,16.76,0.29
1,KXNCAAMBGAME-25NOV24CREIBAY-BAY,0.58,0.60,Baylor,Creighton,h2h,2025-11-24 13:06:13 CST,Baylor Bears,Creighton Bluejays,Baylor Bears,0.6370,0.114634,0.114634,22.926829,39.0,1.0,15.38,1.47
2,KXNCAAMBGAME-25NOV24MASSORST-ORST,0.57,0.59,Oregon St,UMass,h2h,2025-11-24 11:30:00 CST,Oregon St Beavers,Oregon St Beavers,Massachusetts Minutemen,0.6782,0.233810,0.233810,46.761905,82.0,1.0,34.26,8.19
3,KXNCAAMBGAME-25NOV24MICHSDSU-SDSU,0.29,0.30,San Diego St,Michigan,h2h,2025-11-24 21:30:00 CST,San Diego St Aztecs,Michigan Wolverines,San Diego St Aztecs,0.3169,0.031064,0.031064,6.212766,21.0,1.0,13.91,0.16
4,KXNCAAMBGAME-25NOV24HALLNCST-HALL,0.18,0.19,North Carolina St,Seton Hall,h2h,2025-11-24 13:30:00 CST,Seton Hall Pirates,NC State Wolfpack,Seton Hall Pirates,0.2185,0.041104,0.041104,8.220859,45.0,1.0,35.90,1.42


In [625]:
total_loss = np.sum(filtered_winners_df['optimal_bet'])
total_profit = np.sum(filtered_winners_df['profit'])
total_ev = np.sum(filtered_winners_df['ev'])
print(f"{odds_sport} h2h portfolio summary:\n")
print(f"Max Loss: -{total_loss:.2f}")
print(f"Max Profit: {total_profit:.2f}")
print(f"Portfolio EV: {total_ev:.2f}")


cbb h2h portfolio summary:

Max Loss: -90.52
Max Profit: 116.21
Portfolio EV: 11.53


In [626]:
kalshi_cols = ['ticker', 'yes_bid', 'yes_ask', 'team', 'points']
odds_cols = ['market', 'start_time', 'team', 'home_team', 'away_team', 'avg_fair_prb', 'point']

odds_subset = odds_spreads_df[odds_cols].rename(columns={
    'home_team': 'odds_home_team',
    'away_team': 'odds_away_team',
    'team': 'odds_team'
})

kalshi_subset = kalshi_spreads_df[kalshi_cols]

combined_rows = []

for _, kalshi_row in kalshi_subset.iterrows():
    kalshi_home = kalshi_row['team']
    for _, odds_row in odds_subset.iterrows():
        odds_home = odds_row['odds_team']
        if (kalshi_home in odds_home) and (kalshi_row['points'] == odds_row['point']): 
            combined_row = pd.concat([kalshi_row, odds_row])
            combined_rows.append(combined_row)

combined_spreads_df = pd.DataFrame(combined_rows).drop_duplicates(subset='ticker') #only works because oddsapi only pulls odds 
combined_spreads_df = combined_spreads_df.reset_index(drop=True)                    #for only one point line for each bookmaker 



In [627]:
combined_spreads_df

,ticker,yes_bid,yes_ask,team,points,market,start_time,odds_team,odds_home_team,odds_away_team,avg_fair_prb,point
0,KXNCAAMBSPREAD-25NOV24UABSIU-SIU2,0.04,0.96,Southern Illinois,2.5,spreads,2025-11-24 12:30:00 CST,Southern Illinois Salukis,Southern Illinois Salukis,UAB Blazers,0.495067,2.5
1,KXNCAAMBSPREAD-25NOV24UTSAAC-UTSA1,0.41,0.44,UTSA,1.5,spreads,2025-11-24 20:00:00 CST,UTSA Roadrunners,Abilene Christian Wildcats,UTSA Roadrunners,0.500000,1.5
2,KXNCAAMBSPREAD-25NOV24LMUFAU-LMU4,0.20,0.39,Loyola Marymount,4.5,spreads,2025-11-24 19:30:00 CST,Loyola Marymount Lions,Florida Atlantic Owls,Loyola Marymount Lions,0.519600,4.5
3,KXNCAAMBSPREAD-25NOV24OHIOGMU-OHIO6,0.03,0.97,Ohio,6.5,spreads,2025-11-24 17:00:00 CST,Ohio Bobcats,George Mason Patriots,Ohio Bobcats,0.487600,6.5
4,KXNCAAMBSPREAD-25NOV24HOUSYR-SYR14,0.03,0.97,Syracuse,14.5,spreads,2025-11-24 17:00:00 CST,Syracuse Orange,Houston Cougars,Syracuse Orange,0.498180,14.5
5,KXNCAAMBSPREAD-25NOV24TEMUCSD-UCSD1,0.39,0.43,UC San Diego,1.5,spreads,2025-11-24 15:30:00 CST,UC San Diego Tritons,UC San Diego Tritons,Temple Owls,0.486420,1.5
6,KXNCAAMBSPREAD-25NOV24SJUISU-SJU2,0.03,0.97,St. John's,2.5,spreads,2025-11-24 15:30:00 CST,St. John's Red Storm,St. John's Red Storm,Iowa State Cyclones,0.491175,2.5
7,KXNCAAMBSPREAD-25NOV24CREIBAY-CREI5,0.14,0.35,Creighton,5.5,spreads,2025-11-24 13:06:13 CST,Creighton Bluejays,Creighton Bluejays,Baylor Bears,0.500000,5.5
8,KXNCAAMBSPREAD-25NOV24KUND-ND5,0.10,0.31,Notre Dame,5.5,spreads,2025-11-24 14:30:00 CST,Notre Dame Fighting Irish,Kansas Jayhawks,Notre Dame Fighting Irish,0.510500,5.5
